In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
cust_df = pd.read_csv('Customers.csv')
seller_df =pd.read_csv('SELLERS.csv')
prod_df = pd.read_csv('PRODUCTS.csv')
order_df = pd.read_csv('ORDERS.csv')
order_item_df = pd.read_csv('ORDER_ITEMS.csv')
payment_df=pd.read_csv('ORDER_PAYMENTS.csv')
rating_df = pd.read_csv('ORDER_REVIEW_RATINGS.csv')
location_df =pd.read_csv('GEO_LOCATION.csv')

In [3]:
order_df[['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']] = order_df[['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']].apply(lambda x:pd.to_datetime(x,format='%m/%d/%Y %H:%M'))

In [4]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [5]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [6]:
cust_order_df =pd.merge(left=order_df,right=cust_df,on='customer_id',how='left')

In [7]:
cust_order_df['customer_id'].nunique()

99441

In [8]:
# Total number of customers are 99441

In [9]:
cust_order_df['order_id'].nunique()

99441

In [10]:
# Total number of orders are 99441

In [11]:
prod_seller_item_df =pd.merge(left=pd.merge(left=order_item_df,right=seller_df,on='seller_id',how='left'),right=prod_df,on='product_id',how='left')

In [12]:
cust_ord_item_prod_sell = pd.merge(left=cust_order_df,right=prod_seller_item_df,on='order_id',how='left')

In [13]:
cust_ord_item_prod_sell.shape

(113425, 29)

In [14]:
a = (cust_ord_item_prod_sell['order_item_id']*cust_ord_item_prod_sell['price']).sum()

In [15]:
print(f"Total Revenue is {a}")

Total Revenue is 15397738.61


In [16]:
b = cust_ord_item_prod_sell['order_item_id'].sum()

In [17]:
print(f"Total number of products sold are {b}")

Total number of products sold are 134936.0


In [18]:
c= cust_ord_item_prod_sell['product_category_name'].nunique()

In [19]:
print(f"Number of product categories are {c}")

Number of product categories are 71


In [20]:
d = cust_ord_item_prod_sell['seller_id'].nunique()

In [21]:
print(f"Total number of sellers are {d}")

Total number of sellers are 3095


In [22]:
c= cust_ord_item_prod_sell['customer_city'].nunique()
print(f"Total nuber of cities in which products have been sold {c}")

Total nuber of cities in which products have been sold 4119


In [23]:
d = cust_ord_item_prod_sell['customer_state'].nunique()
print(f"Total nuber of states in which products have been sold {d}")

Total nuber of states in which products have been sold 20


In [24]:
e= cust_ord_item_prod_sell['seller_city'].nunique()
print(f"Total nuber of seller cities  {e}")

Total nuber of seller cities  534


In [25]:
f= cust_ord_item_prod_sell['seller_state'].nunique()
print(f"Total nuber of states in which sellers are present {f}")

Total nuber of states in which sellers are present 19


In [26]:
payment_df['payment_type'].value_counts()

payment_type
credit_card    76795
UPI            19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

In [27]:
max_date =cust_ord_item_prod_sell['order_approved_at'].max().date()
min_date = cust_ord_item_prod_sell['order_approved_at'].min().date()
print(f"Data contains orders that have been aproved between {min_date} and {max_date}")

Data contains orders that have been aproved between 2016-09-15 and 2018-09-03


# b. Understanding how many new customers acquired every month


In [28]:
df = cust_ord_item_prod_sell[['customer_unique_id','order_purchase_timestamp']].sort_values(by='order_purchase_timestamp',ignore_index=True)
year = df['order_purchase_timestamp'].dt.year
month = df['order_purchase_timestamp'].dt.month


In [29]:
df.drop_duplicates(subset='customer_unique_id').groupby(by=[year,month]).agg(new_customers=('customer_unique_id','count'))

new_customers
order_purchase_timestamp order_purchase_timestamp               
2016                     9                                     4
                         10                                  321
                         12                                    1
2017                     1                                   764
                         2                                  1752
                         3                                  2636
                         4                                  2352
                         5                                  3596
                         6                                  3139
                         7                                  3894
                         8                                  4184
                         9                                  4130
                         10                                 4470
                         11                                 7304
                         12                                 5487
2018                     1                                  7025
                         2                                  6451
                         3                                  6965
                         4                                  6711
                         5                                  6622
                         6                                  5940
                         7                                  6071
                         8                                  6271
                         9                                     5
                         10                                    1

# Understand the retention of customers on month on month basis

In [30]:
df['order_purchase_date']=pd.to_datetime(df['order_purchase_timestamp'].dt.date)

In [31]:
df.drop(columns='order_purchase_timestamp',inplace=True)

In [32]:
df['year']=df['order_purchase_date'].dt.year
df['month']=df['order_purchase_date'].dt.month

pivot = df.pivot_table(index='customer_unique_id',columns=['year','month'],values='customer_unique_id',aggfunc='count',fill_value=0)

In [33]:
pivot.columns = (df['year'].astype('str')+'-'+df['month'].astype('str')).unique()

In [34]:
pivot.head()

,2016-9,2016-10,2016-12,2017-1,2017-2,2017-3,2017-4,2017-5,2017-6,2017-7,...,2018-1,2018-2,2018-3,2018-4,2018-5,2018-6,2018-7,2018-8,2018-9,2018-10
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
0000b849f77a49e4a4ce2b2a4ca5be3f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
0000f46a3911fa3c0805444483337064,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000f6ccb0745a6a4b88665a16c9f078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0004aac84e0df4da2b147fca70cf8255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
months = pivot.columns
retention_array=[{'YearMonth':months[0],'TotalUserCount':pivot[pivot[months[0]]>0][months[0]].count(),'RetainedUserCount':0 }]
for i in range(len(months)-1):
    retention_data={}
    selected_month=months[i+1]
    prev_month=months[i]
    retention_data['YearMonth']=(selected_month)
    retention_data['TotalUserCount']=pivot[pivot[selected_month]>0][selected_month].count()
    retention_data['RetainedUserCount']=pivot[(pivot[selected_month]>0)&(pivot[prev_month]>0)][selected_month].count()
    retention_array.append(retention_data)

In [36]:
retention_df = pd.DataFrame(retention_array)
retention_df['retention_rate']=retention_df['RetainedUserCount']/retention_df['TotalUserCount']

In [37]:
retention_df

,YearMonth,TotalUserCount,RetainedUserCount,retention_rate
0,2016-9,4,0,0.000000
1,2016-10,321,0,0.000000
2,2016-12,1,0,0.000000
3,2017-1,765,1,0.001307
4,2017-2,1755,3,0.001709
5,2017-3,2642,4,0.001514
6,2017-4,2372,13,0.005481
7,2017-5,3625,14,0.003862
8,2017-6,3180,18,0.005660
9,2017-7,3947,17,0.004307


# d. How the revenues from existing/new customers on month on month basis

In [38]:
df1 = cust_ord_item_prod_sell[['customer_unique_id', 'price', 'order_item_id', 'order_purchase_timestamp']]
df1['revenue']=df1['price']*df1['order_item_id']

In [39]:
df1['year']=df1['order_purchase_timestamp'].dt.year
df1['month']=df1['order_purchase_timestamp'].dt.month

In [40]:
first_buy = df1.groupby('customer_unique_id').agg(min_purchase_date=('order_purchase_timestamp','min')).reset_index()
first_buy['min_year']=first_buy['min_purchase_date'].dt.year
first_buy['min_month']=first_buy['min_purchase_date'].dt.month


In [41]:
df1 = pd.merge(df1, first_buy, on='customer_unique_id')

In [42]:
df1.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp,revenue,year,month,min_purchase_date,min_year,min_month
0,7c396fd4830fd04220f754e42b4e5bff,29.99,1.0,2017-10-02 10:56:00,29.99,2017,10,2017-09-04 11:26:00,2017,9
1,7c396fd4830fd04220f754e42b4e5bff,35.39,1.0,2017-09-04 11:26:00,35.39,2017,9,2017-09-04 11:26:00,2017,9
2,af07308b275d755c9edb36a90c618231,118.70,1.0,2018-07-24 20:41:00,118.70,2018,7,2018-07-24 20:41:00,2018,7
3,3a653a41f6f9fc3d2a113cf8398680e8,159.90,1.0,2018-08-08 08:38:00,159.90,2018,8,2018-08-08 08:38:00,2018,8
4,7c142cf63193a1473d2e66489a9ae977,45.00,1.0,2017-11-18 19:28:00,45.00,2017,11,2017-11-18 19:28:00,2017,11


In [43]:
df1['user_type'] = 'New'
df1.loc[((df1['year'].astype('str') + df1['month'].astype('str')).astype('int')) > ((df1['min_year'].astype('str') + df1['min_month'].astype('str')).astype('int')), 'user_type'] = 'Existing'

In [44]:
df1.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp,revenue,year,month,min_purchase_date,min_year,min_month,user_type
0,7c396fd4830fd04220f754e42b4e5bff,29.99,1.0,2017-10-02 10:56:00,29.99,2017,10,2017-09-04 11:26:00,2017,9,Existing
1,7c396fd4830fd04220f754e42b4e5bff,35.39,1.0,2017-09-04 11:26:00,35.39,2017,9,2017-09-04 11:26:00,2017,9,New
2,af07308b275d755c9edb36a90c618231,118.70,1.0,2018-07-24 20:41:00,118.70,2018,7,2018-07-24 20:41:00,2018,7,New
3,3a653a41f6f9fc3d2a113cf8398680e8,159.90,1.0,2018-08-08 08:38:00,159.90,2018,8,2018-08-08 08:38:00,2018,8,New
4,7c142cf63193a1473d2e66489a9ae977,45.00,1.0,2017-11-18 19:28:00,45.00,2017,11,2017-11-18 19:28:00,2017,11,New


# New customers

In [45]:
df1[df1['user_type']=='New'].groupby(by=['year','month']).agg(Revenue=('revenue','sum')).unstack(level=0).fillna(0)

Revenue                        
year       2016        2017        2018
month                                  
1          0.00   142077.30  1059012.66
2          0.00   269661.74   960925.40
3          0.00   411455.16  1091795.26
4          0.00   397053.94  1106754.59
5          0.00   557221.27  1115520.66
6          0.00   466065.02   950777.68
7          0.00   547631.92   992143.29
8          0.00   643557.67   930672.59
9        435.23   742188.96        0.00
10     56103.79   751727.92        0.00
11         0.00  1158179.02        0.00
12        10.90   796459.82        0.00

# old customers

In [46]:
df1[df1['user_type']=='Existing'].groupby(by=['year','month']).agg(Revenue=('revenue','sum')).unstack(level=0).fillna(0)

Revenue          
year       2017      2018
month                    
1          0.00  13687.25
2        124.92  12146.51
3        561.27  17271.46
4       2282.85  24161.53
5       5166.82  21896.58
6       5583.70  24306.33
7      10403.68  19838.90
8      11778.02  17990.18
9      11701.30    145.00
10     14431.56      0.00
11     18246.05      0.00
12     18582.91      0.00

# 1e. Understand the trends/seasonality of sales, quantity by category, location, month,week, day, time, channel, payment method etc…

In [47]:
cust_payment_df=pd.merge(left=cust_ord_item_prod_sell,right=payment_df,how='left',on='order_id')

In [48]:
cust_payment_df.shape

(118434, 33)

In [49]:
cust_payment_df['order_purchase_date']=cust_payment_df['order_purchase_timestamp'].dt.date

In [50]:
cust_payment_df['order_purchase_date']=cust_payment_df['order_purchase_timestamp'].dt.date
cust_payment_df['order_purchase_date']=pd.to_datetime(cust_payment_df['order_purchase_date'],format='%Y-%m-%d')
cust_payment_df['year']=cust_payment_df['order_purchase_date'].dt.year
cust_payment_df['month']=cust_payment_df['order_purchase_date'].dt.month

In [51]:
cust_payment_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'order_purchase_date', 'year', 'month'],
      dtype='object')

In [52]:
cust_payment_df['Sales']=cust_payment_df['order_item_id']*cust_payment_df['price']


In [53]:
cat_sales_month=cust_payment_df.pivot_table(index='product_category_name',columns=['month'],values='Sales',aggfunc='sum',fill_value=0)

In [54]:
cat_sales_month

month,1,2,3,4,5,6,7,8,9,10,11,12
product_category_name,,,,,,,,,,,,
Agro_Industry_And_Commerce,6079.40,22575.62,7911.49,4500.70,4859.69,4905.80,9912.19,6001.49,2391.99,3350.28,29166.99,5822.60
Air_Conditioning,6826.02,6124.39,7531.19,6202.72,3341.97,10259.18,4282.96,10121.73,1731.05,6869.75,5074.61,668.80
Art,2680.76,1482.70,740.75,2260.18,9746.50,3601.33,2329.94,2035.07,388.54,349.05,150.00,120.60
Arts_And_Craftmanship,0.00,0.00,0.00,15.99,133.78,0.00,824.37,901.76,0.00,0.00,0.00,0.00
Audio,3929.18,3042.09,12031.10,8976.36,4848.76,3575.59,2849.23,3480.50,1272.58,2604.37,4413.09,2706.83
...,...,...,...,...,...,...,...,...,...,...,...,...
Stationery,48736.03,15233.37,20598.53,18277.74,20329.64,18557.83,26695.39,23375.84,6732.20,9596.18,17976.78,27890.89
Tablets_Printing_Image,161.80,1541.87,857.19,1377.78,1663.88,842.00,198.00,648.89,233.90,99.00,350.40,419.40
Telephony,19369.31,26234.35,36605.57,29056.95,32993.08,36409.46,39252.74,61878.11,20349.34,27852.36,30487.83,16074.75


In [55]:
cust_payment_df.pivot_table(index='product_category_name',columns='year',values='Sales',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
product_category_name,,,
Agro_Industry_And_Commerce,0.00,45540.57,61937.67
Air_Conditioning,2935.29,33809.74,32289.34
Art,0.00,10007.64,15877.78
Arts_And_Craftmanship,0.00,173.88,1702.02
Audio,156.99,19083.62,34489.07
...,...,...,...
Stationery,0.00,102738.15,151262.27
Tablets_Printing_Image,0.00,6968.35,1425.76
Telephony,599.48,164156.19,211808.18


In [56]:
cust_payment_df.pivot_table(index='product_category_name',columns=['year','month'],values='Sales',aggfunc=np.sum,fill_value=0).T

product_category_name  Agro_Industry_And_Commerce  Air_Conditioning      Art  \
year month                                                                     
2016 9                                       0.00              0.00     0.00   
     10                                      0.00           2935.29     0.00   
     12                                      0.00              0.00     0.00   
2017 1                                      87.96            813.60     0.00   
     2                                     268.82           3204.30     0.00   
     3                                      81.99           4513.71   279.25   
     4                                       0.00           2781.03   119.90   
     5                                    1579.94           1105.77  6967.65   
     6                                    1390.00           2701.28  1001.20   
     7                                    1180.00           1396.10   386.75   
     8                                     220.00           5885.03   244.70   
     9                                    2391.99           1731.05   388.54   
     10                                   3350.28           3934.46   349.05   
     11                                  29166.99           5074.61   150.00   
     12                                   5822.60            668.80   120.60   
2018 1                                    5991.44           6012.42  2680.76   
     2                                   22306.80           2920.09  1482.70   
     3                                    7829.50           3017.48   461.50   
     4                                    4500.70           3421.69  2140.28   
     5                                    3279.75           2236.20  2778.85   
     6                                    3515.80           7557.90  2600.13   
     7                                    8732.19           2886.86  1943.19   
     8                                    5781.49           4236.70  1790.37   
     9                                       0.00              0.00     0.00   

product_category_name  Arts_And_Craftmanship     Audio      Auto      Baby  \
year month                                                                   
2016 9                                  0.00      0.00      0.00      0.00   
     10                                 0.00    156.99   2028.25   1829.95   
     12                                 0.00      0.00      0.00      0.00   
2017 1                                  0.00      0.00   6695.83   6816.67   
     2                                  0.00    213.60  14792.96   3218.28   
     3                                  0.00   1213.77  16216.61   4068.86   
     4                                  0.00   1211.90  20001.61   4345.25   
     5                                 43.98   2081.18  22932.05  10791.69   
     6                                  0.00   1671.00  33225.29   9134.93   
     7                                129.90    771.69  15294.27  17641.80   
     8                                  0.00   1080.60  50057.16  21182.71   
     9                                  0.00   1272.58  19811.94  26650.74   
     10                                 0.00   2447.38  24463.72  17226.15   
     11                                 0.00   4413.09  38136.26  23690.21   
     12                                 0.00   2706.83  43620.06  20970.67   
2018 1                                  0.00   3929.18  40256.13  33115.26   
     2                                  0.00   2828.49  45085.42  27698.60   
     3                                  0.00  10817.33  47963.81  48907.96   
     4                                 15.99   7764.46  54444.98  57230.70   
     5                                 89.80   2767.58  42774.82  37564.33   
     6                                  0.00   1904.59  54401.87  38068.02   
     7                                694.47   2077.54  46141.29  24842.12   
     8                                901.76   

In [57]:
cust_payment_df.pivot_table(index='product_category_name',columns='month',values='order_item_id',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
product_category_name,,,,,,,,,,,,
Agro_Industry_And_Commerce,28,81,48,12,12,11,33,33,5,5,35,13
Air_Conditioning,41,26,46,42,25,50,24,45,8,36,22,3
Art,20,15,14,27,46,56,25,25,3,3,2,4
Arts_And_Craftmanship,0,0,0,1,3,0,13,9,0,0,0,0
Audio,28,22,51,48,40,33,30,41,11,35,42,18
...,...,...,...,...,...,...,...,...,...,...,...,...
Stationery,480,217,256,239,217,248,319,270,84,123,180,282
Tablets_Printing_Image,3,22,9,15,9,9,2,8,3,1,6,6
Telephony,448,608,575,572,530,436,422,505,193,307,438,315


In [58]:
cust_payment_df.pivot_table(index='product_category_name',columns='year',values='order_item_id',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
product_category_name,,,
Agro_Industry_And_Commerce,0,89,227
Air_Conditioning,18,153,197
Art,0,62,178
Arts_And_Craftmanship,0,3,23
Audio,2,183,214
...,...,...,...
Stationery,0,1109,1806
Tablets_Printing_Image,0,69,24
Telephony,12,2494,2843


In [59]:
cust_payment_df.pivot_table(index='product_category_name',columns=['year','month'],values='order_item_id',aggfunc=np.sum,fill_value=0).T

product_category_name  Agro_Industry_And_Commerce  Air_Conditioning  Art  \
year month                                                                 
2016 9                                          0                 0    0   
     10                                         0                18    0   
     12                                         0                 0    0   
2017 1                                          4                 5    0   
     2                                          9                13    0   
     3                                          2                19    7   
     4                                          0                18    1   
     5                                          4                 9    8   
     6                                          1                14   27   
     7                                          1                 5    3   
     8                                         10                19    4   
     9                                          5                 8    3   
     10                                         5                18    3   
     11                                        35                22    2   
     12                                        13                 3    4   
2018 1                                         24                36   20   
     2                                         72                13   15   
     3                                         46                27    7   
     4                                         12                24   26   
     5                                          8                16   38   
     6                                         10                36   29   
     7                                         32                19   22   
     8                                         23                26   21   
     9                                          0                 0    0   

product_category_name  Arts_And_Craftmanship  Audio  Auto  Baby  \
year month                                                        
2016 9                                     0      0     0     0   
     10                                    0      2    13    17   
     12                                    0      0     0     0   
2017 1                                     0      0    41    57   
     2                                     0      7    98    37   
     3                                     0     15   102    49   
     4                                     0     14   108    58   
     5                                     2     17   164   124   
     6                                     0     11   127    92   
     7                                     1      7   138   146   
     8                                     0      8   394   146   
     9                                     0     11   132   196   
     10                                    0     33   188   149   
     11                                    0     42   292   240   
     12                                    0     18   322   160   
2018 1                                     0     28   303   253   
     2                                     0     15   346   187   
     3                                     0     36   376   244   
     4                                     1     34   393   335   
     5                                     1     23   338   300   
     6                                     0     22   356   278   
     7                                    12     23   375   261   
     8                                     9     33   436   239   
     9                                     0      0     0     0   

product_category_name  Bed_Bath_Table  Books_General_Interest  Books_Imported  \
year month                                                                      
2016 9                              0                       0               0   
     10                            16

# Trends on sales and quantity by Location

In [60]:
cust_payment_df.pivot_table(index='customer_city',columns='month',values='Sales',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
customer_city,,,,,,,,,,,,
Aadityana,475.29,89.90,59.90,0.00,79.89,3398.99,297.10,372.76,0.00,99.00,0.00,0.00
Aambaliyasan,48.90,0.00,0.00,0.00,0.00,0.00,0.00,120.90,0.00,0.00,0.00,0.00
Aantaliya,916.39,1406.40,2125.69,1720.67,1691.69,375.80,2650.10,467.77,149.90,586.35,1173.59,673.12
Aarambhada,329.89,496.60,779.35,560.50,607.57,913.89,247.79,299.00,161.79,461.93,182.98,504.40
Abiramam,0.00,0.00,74.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
Zamin Uthukuli,29.99,0.00,0.00,0.00,0.00,0.00,0.00,78.00,0.00,0.00,0.00,0.00
Ziauddin Pur,469.98,1606.84,1456.23,2478.84,4017.70,2019.24,1785.57,1860.83,569.50,860.36,1283.70,1014.09
Zira,0.00,0.00,139.80,120.00,0.00,124.90,0.00,119.85,0.00,0.00,0.00,0.00


In [61]:
cust_payment_df.pivot_table(index='customer_city',columns='year',values='Sales',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
customer_city,,,
Aadityana,0.0,3813.76,1059.07
Aambaliyasan,0.0,0.00,169.80
Aantaliya,0.0,4347.39,9590.08
Aarambhada,138.0,2714.22,2693.47
Abiramam,0.0,74.90,0.00
...,...,...,...
Zamin Uthukuli,0.0,0.00,107.99
Ziauddin Pur,69.9,8520.98,10832.00
Zira,0.0,124.90,379.65


In [62]:
cust_payment_df.pivot_table(index='customer_city',columns=['year','month'],values='Sales',aggfunc=np.sum,fill_value=0).T

customer_city  Aadityana  Aambaliyasan  Aantaliya  Aarambhada  Abiramam  \
year month                                                                
2016 9              0.00           0.0       0.00        0.00       0.0   
     10             0.00           0.0       0.00      138.00       0.0   
     12             0.00           0.0       0.00        0.00       0.0   
2017 1            109.90           0.0     168.00        0.00       0.0   
     2              0.00           0.0     143.80       66.90       0.0   
     3              0.00           0.0      69.90       65.44      74.9   
     4              0.00           0.0      98.90        0.00       0.0   
     5              0.00           0.0      89.90        0.00       0.0   
     6           3289.00           0.0      49.90      913.89       0.0   
     7             99.10           0.0     729.16      195.89       0.0   
     8            216.76           0.0     414.87      299.00       0.0   
     9              0.00           0.0     149.90      161.79       0.0   
     10            99.00           0.0     586.35      323.93       0.0   
     11             0.00           0.0    1173.59      182.98       0.0   
     12             0.00           0.0     673.12      504.40       0.0   
2018 1            365.39          48.9     748.39      329.89       0.0   
     2             89.90           0.0    1262.60      429.70       0.0   
     3             59.90           0.0    2055.79      713.91       0.0   
     4              0.00           0.0    1621.77      560.50       0.0   
     5             79.89           0.0    1601.79      607.57       0.0   
     6            109.99           0.0     325.90        0.00       0.0   
     7            198.00           0.0    1920.94       51.90       0.0   
     8            156.00         120.9      52.90        0.00       0.0   
     9              0.00           0.0       0.00        0.00       0.0   
     10             0.00           0.0       0.00        0.00       0.0   

customer_city  Abohar  Abrama  Abu Road  Achabal  Achalpur  ...      Yol  \
year month                                                  ...            
2016 9           0.00    0.00      0.00     0.00       0.0  ...     0.00   
     10          0.00  389.00      0.00     0.00       0.0  ...     0.00   
     12          0.00    0.00      0.00     0.00       0.0  ...     0.00   
2017 1           0.00   54.90      0.00     0.00       0.0  ...     0.00   
     2           0.00    0.00      0.00  1511.69       0.0  ...    55.50   
     3           0.00  315.00      0.00    58.98       0.0  ...     0.00   
     4           0.00  349.00      0.00     0.00       0.0  ...    56.85   
     5          31.49    0.00      0.00    54.90       0.0  ...  2227.05   
     6           0.00   20.90      0.00    99.00       0.0  ...   379.79   
     7           0.00    0.00      0.00   129.99       0.0  ...   787.70   
     8           0.00  169.90      0.00    59.90       0.0  ...   175.80   
     9           0.00   69.90      0.00     0.00       0.0  ...    95.76   
     10          0.00   24.00      0.00    55.90       0.0  ...    93.66   
     11          0.00  394.99      0.00  8506.50      39.9  ...   302.88   
     12         43.00  123.98     24.90   367.49       0.0  ...   576.68   
2018 1           0.00  199.00      0.00   218.90       0.0  ...   671.46   
     2           0.00  109.00      0.00   198.90       0.0  ...   497.79   
     3          29.90  198.70      0.00    53.90       0.0  ...   133.90   
     4           0.00    0.00    149.97     0.00       0.0  ...  1543.88   
     5          37.99    0.00      0.00   413.84       0.0  ...   209.98   
     6           0.00  149.90     48.90   550.19       0.0  ...   483.99   
     7           0.00  318.90     70.00   229.90       0.0  ...   201.38   
     8           0.00  199.99      0.00    90.00       0.0  ...   160.70   
     9           0.00    0.00      0.00     0.00       0.0  ...     0.00  

In [63]:
cust_payment_df.pivot_table(index='customer_city',columns='month',values='order_item_id',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
customer_city,,,,,,,,,,,,
Aadityana,8,1,1,0,2,12,5,3,0,1,0,0
Aambaliyasan,1,0,0,0,0,0,0,1,0,0,0,0
Aantaliya,11,15,8,11,11,3,18,5,1,5,13,11
Aarambhada,2,6,9,4,9,3,4,1,4,9,2,8
Abiramam,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Zamin Uthukuli,1,0,0,0,0,0,0,1,0,0,0,0
Ziauddin Pur,8,18,14,16,25,16,31,19,8,12,12,12
Zira,0,0,2,1,0,1,0,1,0,0,0,0


In [64]:
cust_payment_df.pivot_table(index='customer_city',columns='year',values='order_item_id',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
customer_city,,,
Aadityana,0,16,17
Aambaliyasan,0,0,2
Aantaliya,0,49,63
Aarambhada,1,31,29
Abiramam,0,1,0
...,...,...,...
Zamin Uthukuli,0,0,2
Ziauddin Pur,1,80,110
Zira,0,1,4


In [65]:
cust_payment_df.pivot_table(index='customer_city',columns=['year','month'],values='order_item_id',aggfunc=np.sum,fill_value=0).T

customer_city  Aadityana  Aambaliyasan  Aantaliya  Aarambhada  Abiramam  \
year month                                                                
2016 9                 0             0          0           0         0   
     10                0             0          0           1         0   
     12                0             0          0           0         0   
2017 1                 1             0          1           0         0   
     2                 0             0          1           1         0   
     3                 0             0          1           1         1   
     4                 0             0          2           0         0   
     5                 0             0          1           0         0   
     6                11             0          1           3         0   
     7                 2             0          8           3         0   
     8                 1             0          4           1         0   
     9                 0             0          1           4         0   
     10                1             0          5           8         0   
     11                0             0         13           2         0   
     12                0             0         11           8         0   
2018 1                 7             1         10           2         0   
     2                 1             0         14           5         0   
     3                 1             0          7           8         0   
     4                 0             0          9           4         0   
     5                 2             0         10           9         0   
     6                 1             0          2           0         0   
     7                 3             0         10           1         0   
     8                 2             1          1           0         0   
     9                 0             0          0           0         0   
     10                0             0          0           0         0   

customer_city  Abohar  Abrama  Abu Road  Achabal  Achalpur  ...  Yol  \
year month                                                  ...        
2016 9              0       0         0        0         0  ...    0   
     10             0       1         0        0         0  ...    0   
     12             0       0         0        0         0  ...    0   
2017 1              0       1         0        0         0  ...    0   
     2              0       0         0        5         0  ...    1   
     3              0       2         0        2         0  ...    0   
     4              0       1         0        0         0  ...    3   
     5              1       0         0        1         0  ...   12   
     6              0       1         0        1         0  ...    3   
     7              0       0         0        1         0  ...    6   
     8              0       1         0        1         0  ...    1   
     9              0       1         0        0         0  ...    3   
     10             0       1         0        1         0  ...    2   
     11             0       2         0      129         1  ...    4   
     12             1       2         1        4         0  ...    5   
2018 1              0       2         0        2         0  ...    8   
     2              0       1         0        1         0  ...    4   
     3              1       4         0        1         0  ...    2   
     4              0       0         1        0         0  ...    5   
     5              1       0         0        2         0  ...    3   
     6              0       1         1        6         0  ...    3   
     7              0       2         1        1         0  ...    4   
     8              0       1         0        1         0  ...    2   
     9              0       0         0        0         0  ...    0   
     10             0       0         0        0         0  ...    0   

customer_city  Zahirabad  Zai

In [66]:
cust_payment_df.pivot_table(index='customer_state',columns='month',values='Sales',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
customer_state,,,,,,,,,,,,
Andhra Pradesh,726490.39,753000.25,960262.87,952307.08,1082739.84,892339.35,987041.14,1055775.69,489124.74,494061.00,737186.60,513004.37
Arunachal Pradesh,15714.40,17114.16,15785.15,15391.35,16559.93,17415.66,24257.48,19725.12,8579.00,9913.76,13520.19,9291.55
Chhattisgarh,82045.49,81331.61,99848.01,113801.81,125753.18,102435.51,111028.76,95660.01,57801.15,54532.76,77070.31,64506.96
Delhi,40731.80,45551.44,49182.12,60343.56,69367.83,64365.47,70002.61,59097.22,19149.35,36678.43,31796.69,28741.18
Goa,29.50,181.80,0.00,169.99,136.99,55.32,162.30,0.00,0.00,0.00,0.00,0.00
Gujarat,101148.41,100928.86,135288.63,125646.18,137447.23,143453.50,110168.33,120785.16,50358.99,65619.62,101241.02,59212.28
Haryana,35138.19,36407.09,38086.32,40440.00,34846.46,30157.60,40445.11,31802.85,13081.84,18534.19,27323.59,18325.48
Himachal Pradesh,9971.09,8406.30,10849.34,9555.45,16880.73,12002.98,13477.85,10568.81,8589.59,6581.12,8201.55,5582.89
Jammu & Kashmir,27823.13,26382.46,20897.65,24883.60,25963.49,20493.66,21007.70,23713.20,12344.07,10293.05,23125.70,16784.56


In [67]:
cust_payment_df.pivot_table(index='customer_state',columns='year',values='Sales',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
customer_state,,,
Andhra Pradesh,35875.73,4349027.86,5258429.73
Arunachal Pradesh,792.60,84332.39,98142.76
Chhattisgarh,2698.86,497760.66,565356.04
Delhi,1402.36,249116.45,324488.89
Goa,0.00,117.30,618.60
Gujarat,5079.34,573580.31,672638.56
Haryana,911.57,164710.07,198967.08
Himachal Pradesh,36.80,60151.68,60479.22
Jammu & Kashmir,213.80,121754.98,131743.49


In [68]:
cust_payment_df.pivot_table(index='customer_state',columns=['year','month'],values='Sales',aggfunc=np.sum,fill_value=0).T

customer_state  Andhra Pradesh  Arunachal Pradesh  Chhattisgarh     Delhi  \
year month                                                                  
2016 9                  329.44               0.00          0.00      0.00   
     10               35535.39             792.60       2698.86   1402.36   
     12                  10.90               0.00          0.00      0.00   
2017 1                83600.28            2276.77      11749.41   3157.44   
     2               156143.61            3526.56      19792.37   8915.78   
     3               254776.50            5162.73      26225.48  13538.45   
     4               242682.82            4415.78      33523.36  14335.45   
     5               355831.58            7764.51      40233.54  27809.33   
     6               286828.38            7032.10      32398.46  18914.73   
     7               351991.53            9042.54      42169.91  21030.94   
     8               419817.18            4599.50      40455.81  26451.04   
     9               488650.30            8579.00      57801.15  19149.35   
     10              458525.61            9121.16      51833.90  35276.07   
     11              737186.60           13520.19      77070.31  31796.69   
     12              512993.47            9291.55      64506.96  28741.18   
2018 1               642890.11           13437.63      70296.08  37574.36   
     2               596856.64           13587.60      61539.24  36635.66   
     3               705486.37           10622.42      73622.53  35643.67   
     4               709624.26           10975.57      80278.45  46008.11   
     5               726908.26            8795.42      85519.64  41558.50   
     6               605510.97           10383.56      70037.05  45450.74   
     7               635049.61           15214.94      68858.85  48971.67   
     8               635958.51           15125.62      55204.20  32646.18   
     9                  145.00               0.00          0.00      0.00   
     10                   0.00               0.00          0.00      0.00   

customer_state     Goa    Gujarat   Haryana  Himachal Pradesh  \
year month                                                      
2016 9            0.00     105.79      0.00              0.00   
     10           0.00    4973.55    911.57             36.80   
     12           0.00       0.00      0.00              0.00   
2017 1            0.00    9162.97   4408.75            309.79   
     2            0.00   21807.31  12371.32           1678.19   
     3            0.00   36593.04  10681.69           4123.28   
     4            0.00   43688.41  11806.55           2136.80   
     5            0.00   49374.12   9712.14           7941.23   
     6            0.00   46714.87   8056.20           3169.81   
     7          117.30   43050.19  15702.73           5634.82   
     8            0.00   51836.83  15617.16           6239.41   
     9            0.00   50253.20  13081.84           8589.59   
     10           0.00   60646.07  17622.62           6544.32   
     11           0.00  101241.02  27323.59           8201.55   
     12           0.00   59212.28  18325.48           5582.89   
2018 1           29.50   91985.44  30729.44           9661.30   
     2          181.80   79121.55  24035.77           6728.11   
     3            0.00   98695.59  27404.63           6726.06   
     4          169.99   81957.77  28633.45           7418.65   
     5          136.99   88073.11  25134.32           8939.50   
     6           55.32   96738.63  22101.40           8833.17   
     7           45.00   67118.14  24742.38           7843.03   
     8            0.00   68948.33  16185.69           4329.40   
     9            0.00       0.00      0.00              0.00   
     10           0.00       0.00      0.00              0.00   

customer_state  Jammu & Kashmir  Karnataka    Kerala  Madhya Pradesh  \
year month                                                             
2016 9              

In [69]:
cust_payment_df.pivot_table(index='customer_state',columns='month',values='order_item_id',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
customer_state,,,,,,,,,,,,
Andhra Pradesh,6921,7161,8600,8210,9598,8110,8953,9887,3611,4482,6759,4831
Arunachal Pradesh,176,151,150,126,148,119,161,156,54,152,108,83
Chhattisgarh,664,809,782,881,878,858,888,846,397,460,677,493
Delhi,351,411,466,414,476,439,559,471,165,253,338,259
Goa,1,3,0,1,2,1,2,0,0,0,0,0
Gujarat,931,905,1141,907,1093,1099,1093,1006,461,543,892,530
Haryana,303,303,308,329,314,303,363,283,142,127,215,158
Himachal Pradesh,75,113,84,92,109,73,113,103,73,57,62,47
Jammu & Kashmir,270,188,190,214,224,191,217,177,100,90,271,128


In [70]:
cust_payment_df.pivot_table(index='customer_state',columns='year',values='order_item_id',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
customer_state,,,
Andhra Pradesh,321,38903,47899
Arunachal Pradesh,5,735,844
Chhattisgarh,17,3993,4623
Delhi,13,2141,2448
Goa,0,1,9
Gujarat,25,4900,5676
Haryana,9,1415,1724
Himachal Pradesh,2,480,519
Jammu & Kashmir,3,1095,1162


In [71]:
cust_payment_df.pivot_table(index='customer_state',columns=['year','month'],values='order_item_id',aggfunc=np.sum,fill_value=0).T

customer_state  Andhra Pradesh  Arunachal Pradesh  Chhattisgarh  Delhi  Goa  \
year month                                                                    
2016 9                       7                  0             0      0    0   
     10                    313                  5            17     13    0   
     12                      1                  0             0      0    0   
2017 1                     837                 47            72     33    0   
     2                    1316                 35           148    100    0   
     3                    2265                 35           199    146    0   
     4                    1813                 30           271    121    0   
     5                    3101                 57           275    175    0   
     6                    2542                 38           282    149    0   
     7                    3558                 60           382    216    1   
     8                    4110                 41           354    199    0   
     9                    3603                 54           397    165    0   
     10                   4169                147           443    240    0   
     11                   6759                108           677    338    0   
     12                   4830                 83           493    259    0   
2018 1                    6084                129           592    318    1   
     2                    5845                116           661    311    3   
     3                    6335                115           583    320    0   
     4                    6397                 96           610    293    1   
     5                    6497                 91           603    301    2   
     6                    5568                 81           576    290    1   
     7                    5395                101           506    343    1   
     8                    5777                115           492    272    0   
     9                       1                  0             0      0    0   
     10                      0                  0             0      0    0   

customer_state  Gujarat  Haryana  Himachal Pradesh  Jammu & Kashmir  \
year month                                                            
2016 9                3        0                 0                0   
     10              22        9                 2                3   
     12               0        0                 0                0   
2017 1               93       58                 3               28   
     2              163       87                18               33   
     3              331       81                25               56   
     4              279       80                31               68   
     5              396      102                58               84   
     6              369       94                26               73   
     7              458      141                31               86   
     8              410      139                51               81   
     9              458      142                73              100   
     10             521      118                55               87   
     11             892      215                62              271   
     12             530      158                47              128   
2018 1              838      245                72              242   
     2              742      216                95              155   
     3              810      227                59              134   
     4              628      249                61              146   
     5              697      212                51              140   
     6              730      209                47              118   
     7              635      222                82              131   
     8              596      144                52               96   
     9                0        0                 0                0   
     10             

In [72]:
cust_payment_df.pivot_table(index='payment_type',columns='month',values='Sales',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
payment_type,,,,,,,,,,,,
UPI,227477.05,254510.00,259729.68,293323.64,332962.70,248708.80,319225.96,258385.88,133686.54,151289.12,240075.68,160544.26
credit_card,963801.56,976022.60,1244553.16,1217734.84,1349177.66,1161647.27,1209331.69,1301774.84,617687.08,663325.87,933565.55,642221.26
debit_card,11068.99,8368.69,10657.79,12786.32,14460.41,35642.75,39931.19,45270.42,3951.67,5303.88,5763.26,6231.89
not_defined,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
voucher,62324.29,59193.05,76670.83,72800.16,83585.29,71181.94,76732.12,65653.46,42613.93,35544.19,35978.63,34624.35


In [73]:
cust_payment_df.pivot_table(index='payment_type',columns='year',values='Sales',aggfunc=np.sum,fill_value=0)

year,2016,2017,2018
payment_type,,,
UPI,9519.69,1389958.52,1480441.10
credit_card,46386.61,5545429.26,6689027.51
debit_card,209.89,38755.08,160472.29
not_defined,0.00,0.00,0.00
voucher,1598.36,367380.97,347922.91


In [74]:
cust_payment_df.pivot_table(index='payment_type',columns=['year','month'],values='Sales',aggfunc=np.sum,fill_value=0).T

payment_type        UPI  credit_card  debit_card  not_defined   voucher
year month                                                             
2016 9             0.00       165.29        0.00          0.0      0.00
     10         9519.69     46210.42      209.89          0.0   1598.36
     12            0.00        10.90        0.00          0.0      0.00
2017 1         23705.73    116166.21      601.86          0.0   9998.52
     2         53796.44    216047.18     1328.98          0.0  12785.17
     3         72367.12    335463.29     3446.56          0.0  27209.99
     4         86207.39    310843.67     2332.62          0.0  32207.07
     5        129262.84    432088.92     3157.87          0.0  38106.05
     6        104809.44    360444.56     2115.05          0.0  24905.32
     7        114881.58    437840.52     2057.80          0.0  42799.59
     8        128852.07    526121.76     2673.53          0.0  32351.52
     9        133686.54    617521.79     3951.67          0.0  42468.93
     10       141769.43    617115.45     5093.99          0.0  33945.83
     11       240075.68    933565.55     5763.26          0.0  35978.63
     12       160544.26    642210.36     6231.89          0.0  34624.35
2018 1        203771.32    847635.35    10467.13          0.0  52325.77
     2        200713.56    759975.42     7039.71          0.0  46407.88
     3        187362.56    909089.87     7211.23          0.0  49460.84
     4        207116.25    906891.17    10453.70          0.0  40593.09
     5        203699.86    917088.74    11302.54          0.0  45479.24
     6        143899.36    801202.71    33527.70          0.0  46276.62
     7        204344.38    771491.17    37873.39          0.0  33932.53
     8        129533.81    775653.08    42596.89          0.0  33301.94
     9             0.00         0.00        0.00          0.0    145.00
     10            0.00         0.00        0.00          0.0      0.00

In [75]:
cust_payment_df.pivot_table(index='payment_type',columns='month',values='order_item_id',aggfunc=np.sum,fill_value=0)

month,1,2,3,4,5,6,7,8,9,10,11,12
payment_type,,,,,,,,,,,,
UPI,2447,2602,2744,2760,2931,2501,3004,2780,1305,1647,2250,1669
credit_card,8290,8820,10327,9715,11243,9690,10408,11089,4322,5301,8211,5525
debit_card,146,126,130,186,115,272,311,379,49,61,107,71
not_defined,0,0,0,0,0,0,0,0,0,0,0,0
voucher,649,506,724,640,766,727,773,732,394,352,459,383


In [76]:
cust_payment_df.pivot_table(index='payment_type',columns='year',values='order_item_id',aggfunc=np.sum,fill_value=0).T

payment_type,UPI,credit_card,debit_card,not_defined,voucher
year,,,,,
2016,98,358,2,0,22
2017,13580,46069,537,0,3686
2018,14962,56514,1414,0,3397


In [77]:
cust_payment_df.pivot_table(index='payment_type',columns=['year','month'],values='order_item_id',aggfunc=np.sum,fill_value=0).T

payment_type   UPI  credit_card  debit_card  not_defined  voucher
year month                                                       
2016 9           0            4           0            0        0
     10         98          353           2            0       22
     12          0            1           0            0        0
2017 1         301          974          11            0       83
     2         525         1703          15            0      130
     3         797         2635          42            0      256
     4         698         2329          29            0      214
     5        1042         3660          57            0      319
     6         983         3034          31            0      254
     7        1173         4199          24            0      440
     8        1288         4534          42            0      425
     9        1305         4318          49            0      393
     10       1549         4948          59            0      330
     11       2250         8211         107            0      459
     12       1669         5524          71            0      383
2018 1        2146         7316         135            0      566
     2        2077         7117         111            0      376
     3        1947         7692          88            0      468
     4        2062         7386         157            0      426
     5        1889         7583          58            0      447
     6        1518         6656         241            0      473
     7        1831         6209         287            0      333
     8        1492         6555         337            0      307
     9           0            0           0            0        1
     10          0            0           0            0        0

# weekly trends on product and their sales and quantity

In [78]:
cust_payment_df['weekofyear'] =cust_payment_df['order_purchase_date'].dt.isocalendar()['week']

In [79]:
cust_payment_df.pivot_table(index='product_category_name',columns='weekofyear',values='Sales',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
product_category_name,,,,,,,,,,,,,,,,,,,,,
Agro_Industry_And_Commerce,2378.00,1657.44,1054.00,143.97,1746.48,2932.69,15446.97,1413.47,1936.50,1293.00,...,2443.30,0.00,1334.99,2008.00,25386.10,862.90,1193.90,0.00,3429.00,774.70
Air_Conditioning,448.90,2497.60,1220.99,2329.83,527.70,407.90,3184.10,1453.79,1840.38,2768.63,...,0.00,175.90,569.79,1295.87,2641.10,561.85,498.90,0.00,0.00,0.00
Art,389.97,419.97,1168.82,149.00,553.00,954.84,145.99,270.88,119.99,125.00,...,0.00,0.00,0.00,0.00,0.00,150.00,49.90,10.70,60.00,0.00
Arts_And_Craftmanship,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Audio,830.90,684.80,695.50,1404.68,1170.99,39.80,709.70,842.00,1292.90,1812.85,...,611.60,193.58,1169.72,977.40,1541.89,627.29,49.90,561.28,743.89,1351.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stationery,14236.96,15293.14,10690.83,6961.30,4494.04,4945.67,2710.47,2858.04,4548.22,4998.79,...,2491.45,2629.21,4181.38,4327.46,5394.48,6410.21,6538.70,7108.63,5194.38,5349.34
Tablets_Printing_Image,0.00,54.90,54.90,52.00,307.80,304.90,323.68,552.00,205.98,0.00,...,0.00,89.90,0.00,14.90,245.60,69.90,209.70,139.80,0.00,0.00
Telephony,2788.61,4923.40,4080.18,5266.03,5715.54,8614.50,5208.00,7722.20,3956.27,6469.47,...,5618.41,5389.89,5773.21,3830.51,12427.89,9322.50,5993.09,2727.62,1963.72,2314.63


In [80]:
cust_payment_df.pivot_table(index='product_category_name',columns='weekofyear',values='order_item_id',aggfunc=np.sum,fill_value=0)


weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
product_category_name,,,,,,,,,,,,,,,,,,,,,
Agro_Industry_And_Commerce,4,4,12,5,8,21,40,7,10,5,...,2,0,2,5,26,3,4,0,4,4
Air_Conditioning,2,7,8,19,6,3,13,6,7,8,...,0,2,3,8,6,4,2,0,0,0
Art,3,3,9,2,3,9,2,3,2,1,...,0,0,0,0,0,2,1,2,1,0
Arts_And_Craftmanship,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Audio,2,7,7,8,9,2,5,5,9,14,...,8,4,11,6,16,7,1,9,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stationery,133,149,107,68,55,53,47,55,64,65,...,37,27,37,47,57,61,63,69,57,56
Tablets_Printing_Image,0,1,1,1,4,4,6,7,3,0,...,0,1,0,1,4,1,3,2,0,0
Telephony,81,99,107,100,129,147,138,222,96,114,...,60,54,80,66,181,111,95,65,58,63


In [81]:
cust_payment_df.pivot_table(index='customer_city',columns='weekofyear',values='Sales',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
customer_city,,,,,,,,,,,,,,,,,,,,,
Aadityana,35.99,0.00,0.00,329.40,109.90,0.00,0.0,89.90,0.00,59.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
Aambaliyasan,48.90,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
Aantaliya,214.90,225.90,475.59,0.00,189.90,494.70,0.0,282.00,1588.62,594.99,...,389.97,375.46,222.99,105.70,0.00,618.82,245.98,51.97,171.8,53.99
Aarambhada,0.00,99.90,0.00,229.99,52.50,293.79,66.9,0.00,432.35,23.00,...,0.00,79.00,0.00,52.99,0.00,129.99,55.00,170.00,0.0,279.40
Abiramam,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,74.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zamin Uthukuli,0.00,0.00,29.99,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
Ziauddin Pur,0.00,355.19,49.90,0.00,264.79,982.80,69.9,354.24,0.00,463.80,...,60.20,677.95,169.99,95.84,787.89,108.60,533.69,281.90,0.0,89.90
Zira,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00


In [82]:
cust_payment_df.pivot_table(index='customer_city',columns='weekofyear',values='order_item_id',aggfunc=np.sum,fill_value=0)


weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
customer_city,,,,,,,,,,,,,,,,,,,,,
Aadityana,1,0,0,6,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
Aambaliyasan,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aantaliya,2,5,4,0,1,8,0,4,5,2,...,3,2,4,3,0,7,2,3,2,1
Aarambhada,0,1,0,1,1,3,1,0,3,1,...,0,1,0,1,0,1,1,1,0,6
Abiramam,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zamin Uthukuli,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ziauddin Pur,0,5,1,0,3,9,1,7,0,3,...,2,9,1,2,6,4,4,3,0,1
Zira,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
cust_payment_df.pivot_table(index='customer_state',columns='weekofyear',values='Sales',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
customer_state,,,,,,,,,,,,,,,,,,,,,
Andhra Pradesh,131635.24,166374.11,161494.45,178347.11,191135.58,188142.82,177596.70,194041.84,211912.52,218121.31,...,96258.82,115805.63,118350.40,119248.79,298186.32,192712.41,158037.44,127449.09,90188.94,63872.01
Arunachal Pradesh,2019.12,1692.93,3979.47,5895.90,4319.93,4651.70,4187.38,3554.66,5727.77,3575.33,...,3096.70,1308.86,2122.99,1688.33,5757.05,3644.91,2340.13,3955.81,993.58,1059.98
Chhattisgarh,13828.05,17738.25,20185.66,24255.95,15657.12,19662.12,17258.15,20672.04,25109.76,20277.22,...,8917.57,7302.20,10604.45,17167.84,32968.01,19576.10,14557.25,21534.99,10201.90,9783.11
Delhi,5922.44,9237.13,11385.61,10511.17,9990.24,12157.60,8785.30,12819.52,12253.20,11846.57,...,8858.46,6613.17,4899.39,2573.40,15188.65,7929.85,7702.87,7490.54,6476.46,4011.56
Goa,0.00,0.00,29.50,0.00,0.00,36.90,0.00,144.90,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Gujarat,16668.95,27885.38,21860.46,25115.62,27524.88,26750.80,23896.27,19743.02,28110.92,35882.12,...,13455.72,10853.98,15239.84,18391.44,41411.97,26443.50,17125.01,17192.23,8091.19,8450.63
Haryana,3400.90,10251.01,10826.95,8130.55,6773.49,7506.90,8868.59,7912.61,17417.14,11366.70,...,2859.00,2233.43,5715.27,4462.15,12745.93,6682.26,6736.10,3533.49,1682.16,2468.02
Himachal Pradesh,1061.38,4319.55,1018.88,2588.90,2085.03,2469.88,2589.16,1436.37,2880.27,1396.44,...,1471.49,899.78,1282.12,3421.17,1905.98,1888.97,2249.48,871.18,725.55,1149.19
Jammu & Kashmir,4536.74,5451.35,3526.14,9159.51,8700.48,4338.70,8006.77,3242.21,10666.61,2518.37,...,2128.35,2207.91,2800.42,2512.69,13404.64,3449.94,5891.14,3373.23,4369.79,2127.84


In [84]:
cust_payment_df.pivot_table(index='customer_state',columns='weekofyear',values='order_item_id',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
customer_state,,,,,,,,,,,,,,,,,,,,,
Andhra Pradesh,1265,1470,1638,1598,1865,1674,1716,2005,1904,1942,...,924,928,1048,1184,2704,1837,1472,1258,799,664
Arunachal Pradesh,17,30,51,60,33,40,46,30,44,32,...,10,13,19,13,41,30,30,27,9,10
Chhattisgarh,98,182,165,158,144,164,172,276,228,154,...,83,64,96,155,267,179,145,128,77,82
Delhi,65,73,91,78,87,118,99,96,126,122,...,43,64,58,27,147,87,69,60,64,42
Goa,0,0,1,0,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
Gujarat,129,248,196,257,266,208,238,201,243,287,...,113,82,128,187,332,274,160,136,71,85
Haryana,36,103,68,73,54,54,81,72,116,106,...,22,22,38,31,101,56,42,36,30,23
Himachal Pradesh,11,17,10,26,26,23,37,24,27,19,...,9,8,12,13,24,15,12,10,11,7
Jammu & Kashmir,34,42,35,131,58,26,50,34,75,28,...,14,20,24,22,188,38,37,26,34,18


In [85]:
cust_payment_df.pivot_table(index='payment_type',columns='weekofyear',values='Sales',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
payment_type,,,,,,,,,,,,,,,,,,,,,
UPI,39812.76,65577.75,47348.58,49079.27,56147.32,58030.41,45654.25,81754.65,81046.91,62770.96,...,32728.38,31049.12,38813.81,42477.19,97004.68,57320.75,53884.33,39207.60,31378.67,17631.65
credit_card,164823.51,220452.35,220354.02,248073.36,254581.04,249050.03,231322.62,225890.58,284821.03,284441.10,...,127584.27,144582.17,140480.88,155307.97,386153.27,243226.37,188580.38,168027.36,105968.85,89102.48
debit_card,1686.76,2322.30,3514.29,2116.40,2348.82,2861.63,1360.06,1771.92,2543.83,1663.15,...,1140.33,516.98,1141.34,1037.46,2313.61,1661.91,2265.26,1006.27,1060.66,1070.66
not_defined,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
voucher,13492.30,12645.37,14083.51,17017.04,10468.61,13324.89,22622.40,13539.86,11154.97,18272.93,...,9367.93,5599.88,7472.36,4653.60,11627.34,11277.72,8953.52,10905.93,6593.06,4679.01


In [86]:
cust_payment_df.pivot_table(index='payment_type',columns='weekofyear',values='order_item_id',aggfunc=np.sum,fill_value=0)

weekofyear,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
payment_type,,,,,,,,,,,,,,,,,,,,,
UPI,413,652,534,526,621,561,493,890,731,639,...,333,293,367,447,838,585,546,450,309,197
credit_card,1386,1815,1976,2052,2294,2077,2228,2213,2426,2394,...,1075,1059,1201,1359,3475,2201,1604,1384,953,840
debit_card,24,39,43,26,26,29,19,36,46,25,...,17,11,14,13,54,26,26,10,12,13
not_defined,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
voucher,140,101,148,209,116,124,160,122,125,200,...,68,49,80,74,164,154,126,85,48,77


# Trends on Day wise

In [87]:
cust_payment_df['Day'] = cust_payment_df['order_purchase_date'].dt.day_name()

In [88]:
cust_payment_df.pivot_table(index='product_category_name',columns='Day',values='Sales',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
product_category_name,,,,,,,
Agro_Industry_And_Commerce,11007.55,9185.93,30321.95,12164.17,31956.49,8443.41,4398.74
Air_Conditioning,6900.39,18919.98,10475.25,8423.92,8962.39,6642.18,8710.26
Art,4043.62,2782.64,9011.50,2682.69,1984.85,1837.32,3542.80
Arts_And_Craftmanship,362.07,428.78,514.19,41.80,347.37,57.90,123.79
Audio,8406.88,9646.36,8153.13,8685.35,6620.81,6098.12,6119.03
...,...,...,...,...,...,...,...
Stationery,37528.86,43516.11,42197.63,37114.83,38521.97,27167.70,27953.32
Tablets_Printing_Image,378.99,1006.78,1151.29,1847.69,1454.90,932.09,1622.37
Telephony,59990.76,70518.08,70552.97,42896.22,45982.38,46107.04,40516.40


In [89]:
cust_payment_df.pivot_table(index='product_category_name',columns='Day',values='order_item_id',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
product_category_name,,,,,,,
Agro_Industry_And_Commerce,46,36,90,45,67,16,16
Air_Conditioning,36,90,67,41,57,31,46
Art,41,34,31,37,22,24,51
Arts_And_Craftmanship,8,3,4,2,5,1,3
Audio,67,76,48,68,59,34,47
...,...,...,...,...,...,...,...
Stationery,437,510,494,435,430,313,296
Tablets_Printing_Image,5,10,15,13,20,12,18
Telephony,851,854,915,812,749,600,568


In [90]:
cust_payment_df.pivot_table(index='customer_city',columns='Day',values='Sales',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
customer_city,,,,,,,
Aadityana,326.66,294.68,98.99,285.30,0.00,149.80,3717.40
Aambaliyasan,0.00,0.00,0.00,0.00,120.90,48.90,0.00
Aantaliya,2525.84,2474.90,1686.66,1868.64,1181.68,1320.16,2879.59
Aarambhada,987.10,1172.49,1199.54,905.59,459.83,352.28,468.86
Abiramam,0.00,0.00,0.00,74.90,0.00,0.00,0.00
...,...,...,...,...,...,...,...
Zamin Uthukuli,0.00,0.00,29.99,78.00,0.00,0.00,0.00
Ziauddin Pur,2098.87,2725.73,2916.97,2781.18,3428.38,3769.04,1702.71
Zira,124.90,259.80,119.85,0.00,0.00,0.00,0.00


In [91]:
cust_payment_df.pivot_table(index='customer_city',columns='Day',values='order_item_id',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
customer_city,,,,,,,
Aadityana,2,5,2,4,0,2,18
Aambaliyasan,0,0,0,0,1,1,0
Aantaliya,16,23,16,19,16,10,12
Aarambhada,11,17,12,5,5,5,6
Abiramam,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
Zamin Uthukuli,0,0,1,1,0,0,0
Ziauddin Pur,24,21,28,19,40,41,18
Zira,1,3,1,0,0,0,0


In [92]:
cust_payment_df.pivot_table(index='customer_state',columns='Day',values='Sales',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
customer_state,,,,,,,
Andhra Pradesh,1567496.37,1593452.01,1512234.96,1482531.00,1389563.33,1031647.22,1066408.43
Arunachal Pradesh,31329.07,32386.16,28458.87,20526.27,23606.34,19850.99,27110.05
Chhattisgarh,163031.44,167585.80,175079.32,151276.38,162066.24,112902.83,133873.55
Delhi,86270.92,82932.32,84492.46,84620.49,88878.10,76181.26,71632.15
Goa,45.00,39.99,277.32,36.90,19.90,287.29,29.50
Gujarat,203815.08,183439.22,198004.33,183094.76,185042.96,146445.56,151456.30
Haryana,69431.30,57138.55,51825.36,54552.80,49406.33,35652.26,46582.12
Himachal Pradesh,18118.43,15751.14,16504.42,16495.39,29172.72,12474.15,12151.45
Jammu & Kashmir,40376.58,39715.28,34415.49,46019.64,38211.67,30160.34,24813.27


In [93]:
cust_payment_df.pivot_table(index='customer_state',columns='Day',values='order_item_id',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
customer_state,,,,,,,
Andhra Pradesh,14045,14605,13630,13264,12305,9012,10262
Arunachal Pradesh,343,274,228,194,189,168,188
Chhattisgarh,1294,1337,1468,1301,1298,906,1029
Delhi,709,674,704,755,679,530,551
Goa,1,1,3,1,1,2,1
Gujarat,1712,1698,1698,1551,1505,1235,1202
Haryana,604,490,431,467,459,355,342
Himachal Pradesh,162,158,162,146,164,94,115
Jammu & Kashmir,366,304,310,418,323,323,216


In [94]:
cust_payment_df.pivot_table(index='payment_type',columns='Day',values='Sales',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
payment_type,,,,,,,
UPI,508825.52,505603.10,475566.77,494167.13,417564.80,231924.27,246267.72
credit_card,1960712.30,1948993.40,1882876.32,1782479.47,1818867.58,1409898.50,1477015.81
debit_card,26216.70,31580.98,30211.40,40655.72,24957.77,19529.23,26285.46
not_defined,0.00,0.00,0.00,0.00,0.00,0.00,0.00
voucher,112760.91,106492.65,124862.34,121337.76,97802.12,80463.99,73182.47


In [95]:
cust_payment_df.pivot_table(index='payment_type',columns='Day',values='order_item_id',aggfunc=np.sum,fill_value=0).iloc[:,[1,5,6,4,0,2,3]]

Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
payment_type,,,,,,,
UPI,4904,5186,4810,4887,4063,2328,2462
credit_card,16585,16379,15870,14953,14606,11523,13025
debit_card,272,337,293,354,279,184,234
not_defined,0,0,0,0,0,0,0
voucher,1087,1196,1114,1115,1029,849,715


# Trends on hour wise 

In [96]:
cust_payment_df['order_purchase_hour'] = cust_payment_df['order_purchase_timestamp'].dt.hour

In [97]:
bins = [0, 6, 12, 18, 24]
labels = ['00:00-05:59', '06:00-11:59', '12:00-17:59', '18:00-23:59']


In [98]:
cust_payment_df['Time_bin'] = pd.cut(cust_payment_df['order_purchase_hour'], bins, labels=labels, right=False)


In [99]:
cust_payment_df['Time_bin'].value_counts()

Time_bin
12:00-17:59    45919
18:00-23:59    40385
06:00-11:59    26429
00:00-05:59     5701
Name: count, dtype: int64

In [100]:
cust_payment_df.groupby(by='Time_bin').agg(sales=('Sales',np.sum))

,sales
Time_bin,
00:00-05:59,700431.15
06:00-11:59,3566767.89
12:00-17:59,6416412.25
18:00-23:59,5393760.84


In [101]:
cust_payment_df.groupby(by='Time_bin').agg(total_qty=('order_item_id',np.sum))

,total_qty
Time_bin,
00:00-05:59,6716.0
06:00-11:59,31646.0
12:00-17:59,55126.0
18:00-23:59,47157.0


# f. Popular Products by month, seller, state, category.


In [102]:
products = cust_payment_df.groupby(by=['month','product_id']).agg(Qty=('order_item_id',np.sum)).sort_values(by=['month','Qty'],ascending=[True,False])


# Top 5 products on each month

In [103]:
products.groupby(level=0).head(5)


Qty
month product_id                             
1     aca2eb7d00ea1a7b8ebd4e68314663af  139.0
      37eb69aca8718e843d897aa7b82f462d  120.0
      a62e25e09e05e6faf31d90c6ec1aa3d1  118.0
      36f60d45225e60c7da4558b070ce4b60  109.0
      54d9ac713e253fa1fae9c8003b011c2a   78.0
2     ee3d532c8a438679776d222e997606b3  215.0
      44a5d24dd383324a421569ca697b13c2  107.0
      e53e557d5a159f5aa2c5e995dfdf244b  105.0
      3dd2a17168ec895c781a9191c1e95ad7   79.0
      368c6c730842d78016ad823897a372db   78.0
3     aca2eb7d00ea1a7b8ebd4e68314663af   73.0
      dbb67791e405873b259e4656bf971246   66.0
      437c05a395e9e47f9762e677a7068ce7   62.0
      3dd2a17168ec895c781a9191c1e95ad7   57.0
      53759a2ecddad2bb87a079a1f1519f73   55.0
4     aca2eb7d00ea1a7b8ebd4e68314663af  109.0
      d34c07a2d817ac73f4caf8c574215fed  106.0
      53b36df67ebb7c41585e8d54d6772e08   99.0
      d1c427060a0f73f6b889a5c7c61f2ac4   67.0
      422879e10f46682990de24d770e7f83d   62.0
5     53b36df67ebb7c41585e8d54d6772e08  124.0
      aca2eb7d00ea1a7b8ebd4e68314663af  111.0
      422879e10f46682990de24d770e7f83d  108.0
      99a4788cb24856965c36a24e339b6058   96.0
      4e53a453045707bbc5febcf5f32097ac   84.0
6     99a4788cb24856965c36a24e339b6058   89.0
      42a2c92a0979a949ca4ea89ec5c7b934   83.0
      a92930c327948861c015c919a0bcb4a8   44.0
      ac1ad58efc1ebf66bfadc09f29bdedc0   44.0
      19c91ef95d509ea33eda93495c4d3481   43.0
7     270516a3f41dc035aa87d220228f844c  145.0
      99a4788cb24856965c36a24e339b6058   75.0
      32e18e89237933ebdaaebd78a27e7fa1   69.0
      05b515fdc76e888aada3c6d66c201dff   65.0
      f1c7f353075ce59d8a6f3cf58f419c9c   65.0
8     9571759451b1d780ee7c15012ea109d4  210.0
      e7cc48a9daff5436f63d3aad9426f28b  140.0
      422879e10f46682990de24d770e7f83d   72.0
      5ddab10d5e0a23acb99acf56b62b3276   63.0
      ebf9bc6cd600eadd681384e3116fda85   63.0
9     0554911df28fda9fd668ce5ba5949695   57.0
      d5991653e037ccb7af6ed7d94246b249   57.0
      b0961721fd839e9982420e807758a2a6   55.0
      422879e10f46682990de24d770e7f83d   44.0
      53759a2ecddad2bb87a079a1f1519f73   38.0
10    03e1c946c0ddfc58724ff262aef08dff   78.0
      422879e10f46682990de24d770e7f83d   73.0
      b532349fe46b38fbc7bb3914c1bdae07   56.0
      368c6c730842d78016ad823897a372db   54.0
      54d7ad85b648d3cb2b7069ed446d7f35   52.0
11    389d119b48cf3043d311335e499d9c6b  137.0
      422879e10f46682990de24d770e7f83d  130.0
      89b190a046022486c635022524a974a8  120.0
      53759a2ecddad2bb87a079a1f1519f73  114.0
      368c6c730842d78016ad823897a372db   94.0
12    422879e10f46682990de24d770e7f83d  166.0
      368c6c730842d78016ad823897a372db   69.0
      aca2eb7d00ea1a7b8ebd4e68314663af   63.0
      53759a2ecddad2bb87a079a1f1519f73   58.0
      4c2394abfbac7ff59ec7a420918562fa   53.0

# Top sellers in terms of total quantity and their products sold in descending order

In [104]:
sellers = cust_payment_df.groupby(by='seller_id').agg(Total_Qty=('order_item_id',np.sum)) \
.sort_values(by='Total_Qty',ascending=False)
unique_seller = pd.DataFrame(sellers.index)

In [105]:
seller_products =cust_payment_df.groupby(by=['seller_id','product_id']).agg(prod_qty=('order_item_id',np.sum)).reset_index(level=1)\
.sort_values(by='prod_qty',ascending=False)
b =seller_products.groupby(by='seller_id').head(5)

In [106]:
pd.merge(left=unique_seller,right=b,on='seller_id',how='inner')

,seller_id,product_id,prod_qty
0,1f50f920176fa81dab994f9023523100,422879e10f46682990de24d770e7f83d,820.0
1,1f50f920176fa81dab994f9023523100,53759a2ecddad2bb87a079a1f1519f73,561.0
2,1f50f920176fa81dab994f9023523100,368c6c730842d78016ad823897a372db,559.0
3,1f50f920176fa81dab994f9023523100,389d119b48cf3043d311335e499d9c6b,553.0
4,1f50f920176fa81dab994f9023523100,b0961721fd839e9982420e807758a2a6,181.0
...,...,...,...
10367,63bc58679c44e8a998792119fd7ad23e,50cada320c34c3d63fa8598fdda8e06e,1.0
10368,63704069d9bd3a75c1cf59babe56004a,436f1aaf28631abf937c44f695d3c350,1.0
10369,63191ead8d3b2f937cd1996fdcbbad20,c8e167a128928bf9c4e5ab674e78d2c2,1.0
10370,d63f306de130e0391d38a792541723b8,2031ee6574c322f4e3be624d8433052d,1.0


# Top 5 products statewise

In [107]:
state =cust_payment_df.groupby(by=['customer_state','product_id']).agg(prod_qty=('order_item_id',np.sum))\
.sort_values(by='prod_qty',ascending=False)
state.groupby(by='customer_state').head(5)

prod_qty
customer_state product_id                                
Andhra Pradesh 422879e10f46682990de24d770e7f83d     492.0
               aca2eb7d00ea1a7b8ebd4e68314663af     430.0
               99a4788cb24856965c36a24e339b6058     357.0
               389d119b48cf3043d311335e499d9c6b     304.0
               368c6c730842d78016ad823897a372db     299.0
...                                                   ...
Goa            99ba5fbf3ae895fe3a9289acd0a5eb91       1.0
               a29c670cb6c4fb685d04410dbbd6d5b2       1.0
               a50acd33ba7a8da8e9db65094fa990a4       1.0
               b99e4f4fa3f421e0ffbd512d9f152dec       1.0
               bfc275f6de912665e4dcd8da32f43c10       1.0

[100 rows x 1 columns]

# Top 5 categories

In [108]:
categories =cust_payment_df.groupby(by=['product_category_name','product_id']).agg(prod_qty=('order_item_id',np.sum))\
.sort_values(by=['product_category_name','prod_qty'],ascending=[True,False])
categories.groupby(level=0).head(5)

prod_qty
product_category_name      product_id                                
Agro_Industry_And_Commerce 11250b0d4b709fee92441c5f34122aed      56.0
                           c183fd5d2abf05873fa6e1014ed9e06c      21.0
                           672e757f331900b9deea127a2a7b79fd      20.0
                           423a6644f0aa529e8828ff1f91003690      19.0
                           980ecbcc15fe174ec1e5757c4d75b1bf      17.0
...                                                               ...
Watches_Gifts              a62e25e09e05e6faf31d90c6ec1aa3d1     369.0
                           53b36df67ebb7c41585e8d54d6772e08     363.0
                           e0d64dcfaa3b6db5c54ca298ae101d05     206.0
                           a92930c327948861c015c919a0bcb4a8     168.0
                           461f43be3bdf8844e65b62d9ac2c7a5a     150.0

[348 rows x 1 columns]

# g. Popular categories by state, month


In [109]:
state =cust_payment_df.groupby(by=['customer_state','product_category_name']).agg(prod_cat_qty=('order_item_id',np.sum))\
.sort_values(by='prod_cat_qty',ascending=False)
state.groupby(by='customer_state').head(5)

prod_cat_qty
customer_state product_category_name                        
Andhra Pradesh Bed_Bath_Table                         9439.0
               Furniture_Decor                        7553.0
               Health_Beauty                          7299.0
               Computers_Accessories                  6516.0
               Sports_Leisure                         6370.0
...                                                      ...
Goa            Construction_Tools_Construction           1.0
               Auto                                      1.0
               Fashion_Male_Clothing                     1.0
               Musical_Instruments                       1.0
               Housewares                                1.0

[100 rows x 1 columns]

In [110]:
month_cat =cust_payment_df.groupby(by=['month','product_category_name']).agg(prod_cat_qty=('order_item_id',np.sum))\
.sort_values(by=['month','prod_cat_qty'],ascending=[True,False])
month_cat.groupby(level=0).head(5)

prod_cat_qty
month product_category_name              
1     Bed_Bath_Table               1277.0
      Furniture_Decor              1176.0
      Computers_Accessories         942.0
      Sports_Leisure                898.0
      Health_Beauty                 821.0
2     Computers_Accessories        1552.0
      Bed_Bath_Table               1061.0
      Health_Beauty                 982.0
      Sports_Leisure                934.0
      Furniture_Decor               924.0
3     Bed_Bath_Table               1402.0
      Furniture_Decor              1350.0
      Computers_Accessories        1201.0
      Sports_Leisure               1177.0
      Health_Beauty                1004.0
4     Bed_Bath_Table               1349.0
      Furniture_Decor              1120.0
      Health_Beauty                1023.0
      Sports_Leisure                952.0
      Housewares                    853.0
5     Bed_Bath_Table               1462.0
      Housewares                   1319.0
      Health_Beauty                1267.0
      Furniture_Decor              1221.0
      Computers_Accessories        1087.0
6     Bed_Bath_Table               1501.0
      Health_Beauty                1334.0
      Housewares                   1316.0
      Furniture_Decor               869.0
      Sports_Leisure                858.0
7     Bed_Bath_Table               1519.0
      Health_Beauty                1495.0
      Housewares                   1239.0
      Furniture_Decor              1096.0
      Sports_Leisure               1055.0
8     Bed_Bath_Table               1525.0
      Health_Beauty                1379.0
      Furniture_Decor              1278.0
      Housewares                   1183.0
      Sports_Leisure               1019.0
9     Bed_Bath_Table                674.0
      Sports_Leisure                543.0
      Furniture_Decor               474.0
      Health_Beauty                 439.0
      Computers_Accessories         357.0
10    Bed_Bath_Table                828.0
      Furniture_Decor               664.0
      Sports_Leisure                575.0
      Computers_Accessories         482.0
      Health_Beauty                 480.0
11    Furniture_Decor              1333.0
      Bed_Bath_Table               1254.0
      Garden_Tools                  771.0
      Sports_Leisure                712.0
      Computers_Accessories         696.0
12    Bed_Bath_Table                687.0
      Sports_Leisure                597.0
      Furniture_Decor               550.0
      Garden_Tools                  536.0
      Health_Beauty                 527.0

# h. List top 10 most expensive products sorted by price


In [111]:
cust_payment_df[['product_id','price']].groupby(by='product_id').agg(Avg_price=('price',np.mean)). \
sort_values(by='Avg_price',ascending=False).head(10)

,Avg_price
product_id,
489ae2aa008f021502940f251d4cce7f,6735.00
69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
1bdf5e6731585cf01aa8169c7028d6ad,6499.00
a6492cc69376c469ab6f61d8f44de961,4799.00
c3ed642d592594bb648ff4a04cee2747,4690.00
259037a6a41845e455183f89c5035f18,4590.00
a1beef8f3992dbd4cd8726796aa69c53,4399.87
6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
6902c1962dd19d540807d0ab8fade5c6,3999.90


# 2. Performing Customers/sellers Segmentation

# a. Divide the customers into groups based on the revenue generated 


In [112]:
a =cust_payment_df.groupby(by='customer_unique_id').agg(cust_total=('Sales',np.sum))


In [113]:
cust_payment_df = pd.merge(left=cust_payment_df,right=a,on='customer_unique_id',how='inner')

In [114]:
bins=[0,200,500,1000,cust_payment_df['cust_total'].max()]
lables=['Low Revenue','Mid Revenue','High Revenue','Premium']

In [115]:
cust_payment_df['customer_segment'] = pd.cut(cust_payment_df['cust_total'],bins=bins,labels=lables)

In [116]:
cust_payment_df.groupby(by='customer_segment')['customer_unique_id'].nunique()

customer_segment
Low Revenue     76898
Mid Revenue     13285
High Revenue     3650
Premium          1587
Name: customer_unique_id, dtype: int64

# seller segmentation

In [117]:
seller_agg =cust_payment_df.groupby(by='seller_id').agg(seller_total=('Sales',np.sum))


In [118]:
cust_payment_df = pd.merge(left=cust_payment_df,right=seller_agg,on='seller_id',how='inner')

In [119]:
bins=[0,500,1000,5000,cust_payment_df['seller_total'].max()]
lables=['Bronze','Silver','Gold','Platinum']

In [120]:
cust_payment_df['seller_segmentation'] = pd.cut(cust_payment_df['seller_total'],bins=bins,labels=lables)

In [121]:
cust_payment_df.groupby(by='seller_segmentation')['seller_id'].nunique()

seller_segmentation
Bronze      1159
Silver       413
Gold         870
Platinum     653
Name: seller_id, dtype: int64

# 3. Cross-Selling (Which products are selling together)


In [122]:
orders=cust_payment_df[['order_id','product_category_name']].drop_duplicates()

In [123]:
count_of_products=orders.groupby(by=['order_id']).agg(Total_products=('product_category_name','count')).sort_values(by=['Total_products'],ascending=False).reset_index()

In [124]:
count = count_of_products[count_of_products['Total_products']>1]

In [125]:
count

,order_id,Total_products
0,d4bec1a24c97bd17be18d77297a0f6a0,3
1,2f8f31eb2f7b6572836d662a6625c8e4,3
2,ab1a70d5cd14419a6f2c2aca5ee8a79b,3
3,e8c92cfd87f5f0c6d2fc5bc1df5f02b4,3
4,a98012aa8c697f2bcff9f8c2183e0f2b,3
...,...,...
721,673242a6057d4287cb24379d405cf5ac,2
722,6efae6c7b38cde4c5833376accaedb02,2
723,c5d41b216e4b42500c5da2be17a74065,2
724,3b2285d4d80f742f34cc959e041722a9,2


In [126]:
table1 = pd.merge(left=orders,right=count,on='order_id',how='inner')

In [127]:
table1.columns

Index(['order_id', 'product_category_name', 'Total_products'], dtype='object')

In [128]:
pivot_tab = table1.pivot_table(index='order_id',columns='product_category_name',values='product_category_name',aggfunc='count',fill_value=0)

In [129]:
pivot_tab.shape

(726, 60)

In [130]:
array=[]
for x in range (pivot_tab.columns.size):
    for y in range (pivot_tab.columns.size):
        
        if pivot_tab.columns[x] != pivot_tab.columns[y]:
            
            dict_1={}
            Assosciated_count = pivot_tab[(pivot_tab[pivot_tab.columns[x]]>0)&(pivot_tab[pivot_tab.columns[y]]>0)][pivot_tab.columns[x]].count()
            dict_1['Associated_products']= f"{pivot_tab.columns[x]} and {pivot_tab.columns[y]}"
            dict_1['Associated_percentage']=Assosciated_count/(pivot_tab.shape[0])*100
            array.append(dict_1)
        
            
            
            
            

In [131]:
association_df=pd.DataFrame(array)

In [170]:
association_df.sort_values(by=['Associated_percentage'],ascending=False,ignore_index=True).head(20)

,Associated_products,Associated_percentage
0,Bed_Bath_Table and Furniture_Decor,9.641873
1,Furniture_Decor and Bed_Bath_Table,9.641873
2,Home_Confort and Bed_Bath_Table,5.922865
3,Bed_Bath_Table and Home_Confort,5.922865
4,Housewares and Furniture_Decor,3.305785
5,Furniture_Decor and Housewares,3.305785
6,Baby and Cool_Stuff,2.754821
7,Housewares and Bed_Bath_Table,2.754821
8,Cool_Stuff and Baby,2.754821
9,Bed_Bath_Table and Housewares,2.754821


In [133]:
array_1=[]
for (index,row) in pivot_tab.iterrows():
   
    list1=[]
    dict2={}
    
    for column in pivot_tab.columns:
        
        if row[column]>=1:
            list1.append(column)
            
        
    dict2['Assosciated_products']  =" & ".join(list1)         
    
    array_1.append(dict2)

In [134]:
z = pd.DataFrame(array_1)


In [135]:
z.value_counts()

Assosciated_products            
Bed_Bath_Table & Furniture_Decor    67
Bed_Bath_Table & Home_Confort       42
Furniture_Decor & Housewares        22
Bed_Bath_Table & Housewares         20
Baby & Toys                         19
                                    ..
Consoles_Games & Cool_Stuff          1
Consoles_Games & Housewares          1
Consoles_Games & Pet_Shop            1
Consoles_Games & Sports_Leisure      1
Toys & Watches_Gifts                 1
Name: count, Length: 244, dtype: int64

In [136]:
(z.value_counts().divide(z.value_counts().sum())*100)

Assosciated_products            
Bed_Bath_Table & Furniture_Decor    9.228650
Bed_Bath_Table & Home_Confort       5.785124
Furniture_Decor & Housewares        3.030303
Bed_Bath_Table & Housewares         2.754821
Baby & Toys                         2.617080
                                      ...   
Consoles_Games & Cool_Stuff         0.137741
Consoles_Games & Housewares         0.137741
Consoles_Games & Pet_Shop           0.137741
Consoles_Games & Sports_Leisure     0.137741
Toys & Watches_Gifts                0.137741
Name: count, Length: 244, dtype: float64

# 4. Payment Behaviour

# a. How customers are paying

In [137]:
payment_behaviour = pd.merge(left=cust_ord_item_prod_sell,right=payment_df,on='order_id',how='left')

In [138]:
payment_behaviour.groupby(by='payment_type').agg(Total_payment_amount=('payment_value',np.sum),Total_orders=('order_id','nunique'),\
                                              Total_customers=('customer_unique_id','nunique'))

,Total_payment_amount,Total_orders,Total_customers
payment_type,,,
UPI,4086820.71,19784,19290
credit_card,15694885.84,76505,74092
debit_card,256417.92,1528,1513
not_defined,0.00,3,3
voucher,432602.19,3866,3760


In [139]:
a =payment_behaviour.groupby(by='payment_type').agg(Total_percentage_of_customers=('customer_unique_id','nunique'))


In [140]:
(a.divide(a.sum())*100).sort_values(by='Total_percentage_of_customers',ascending=False)

,Total_percentage_of_customers
payment_type,
credit_card,75.099840
UPI,19.552393
voucher,3.811146
debit_card,1.533581
not_defined,0.003041


# 5. Customer satisfaction towards category & product



# a. Which categories (top 10) are maximum rated & minimum rated?

In [141]:
rate =pd.merge(left=pd.merge(left=pd.merge(left=order_df,right=order_item_df,on='order_id',how='left'),right=prod_df,on='product_id',how='left'),\
               right=rating_df,on='order_id',how='left')
               

In [142]:
rate.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'review_id', 'review_score',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object')

In [143]:
rate['review_score'] = rate['review_score'].astype('int')

In [144]:
rate[['product_category_name','review_score']].groupby(by='product_category_name').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=False).head(10)

,mean_score
product_category_name,
Cds_Dvds_Musicals,4.642857
Fashion_Childrens_Clothes,4.500000
Books_General_Interest,4.439421
Books_Imported,4.400000
Costruction_Tools_Tools,4.359223
Books_Technical,4.338290
Luggage_Accessories,4.309524
Food_Drink,4.303571
Small_Appliances_Home_Oven_And_Coffee,4.302632


In [145]:
rate[['product_category_name','review_score']].groupby(by='product_category_name').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=True).head(10)

,mean_score
product_category_name,
Security_And_Services,2.500000
Diapers_And_Hygiene,3.256410
Home_Comfort_2,3.366667
Office_Furniture,3.484421
Fashion_Male_Clothing,3.621212
Fixed_Telephony,3.671698
Party_Supplies,3.767442
Fashio_Female_Clothing,3.780000
La_Cuisine,3.785714


# b. Which products (top10) are maximum rated & minimum rated?
 

In [146]:
rate[['product_id','review_score']].groupby(by='product_id').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=False).head(10)

,mean_score
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
86743ff92eee3d16b7df59cddd583b8c,5.0
868ceb027ab706a4dee42e2220006b85,5.0
868969d3a93aeeab7bfcd4fc3d3d65de,5.0
868766ed2172644fdd977d6bd395a107,5.0
8684bd8f93b4f4038d07188a23811e93,5.0
867c96d2bb67aba6500a4c509cf76072,5.0
867b820367ec206b38a357f2c12454b7,5.0
867901d7e8488fb97f1fb538c09d476e,5.0


In [147]:
rate[['product_id','review_score']].groupby(by='product_id').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=True).head(10)

,mean_score
product_id,
592cc6634d2c783d297effc0b828bc37,1.0
28c95d7653177449fd4b6fc7adb63ce9,1.0
28d6d6a10ceb0636ae572e7efafef612,1.0
28d7da9ebe96bfda54aed060b68b23a1,1.0
8c4e5e50a08a443b7fbad34ad7587625,1.0
8c4dc358af6e14174f2114be62c18f37,1.0
8c4456c061013adccb1ba4cbce2d0365,1.0
8c35dc99b37ba51780800ba049ed3616,1.0
28fe66c8a81afe9be7829f40a5fae9ee,1.0


# c. Average rating by location, seller, product, category, month etc

In [148]:
df_5 = pd.merge(left=cust_ord_item_prod_sell,right=rating_df,on='order_id',how='left')

In [149]:
df_5.groupby(by='customer_city').agg(mean_score=('review_score',np.mean))

,mean_score
customer_city,
Aadityana,3.550000
Aambaliyasan,3.500000
Aantaliya,3.920000
Aarambhada,3.568627
Abiramam,5.000000
...,...
Zamin Uthukuli,4.000000
Ziauddin Pur,4.012422
Zira,4.500000


In [150]:
df_5.groupby(by='seller_id').agg(mean_score=('review_score',np.mean))

,mean_score
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3.666667
001cca7ae9ae17fb1caed9dfb1094831,3.874477
001e6ad469a905060d959994f1b41e4f,1.000000
002100f778ceb8431b7a1020ff7ab48f,3.982143
003554e2dce176b5555353e4f3555ac8,5.000000
...,...
ffcfefa19b08742c5d315f2791395ee5,1.000000
ffdd9f82b9a447f6f8d4b91554cc7dd3,4.250000
ffeee66ac5d5a62fe688b9d26f83f534,4.214286


In [151]:
df_5[['product_id','review_score']].groupby(by='product_id').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=False)

,mean_score
product_id,
00066f42aeeb9f3007548bb9d3f33c38,5.0
86743ff92eee3d16b7df59cddd583b8c,5.0
868ceb027ab706a4dee42e2220006b85,5.0
868969d3a93aeeab7bfcd4fc3d3d65de,5.0
868766ed2172644fdd977d6bd395a107,5.0
...,...
6b3e612bee768d758d981f0d7992e07e,1.0
1890c01a38d17958d79fe473d451dc9c,1.0
94982c575245c1664b51851ec653a79e,1.0


In [152]:
df_5[['product_category_name','review_score']].groupby(by='product_category_name').agg(mean_score=('review_score',np.mean))\
.sort_values(by='mean_score',ascending=False)

,mean_score
product_category_name,
Cds_Dvds_Musicals,4.642857
Fashion_Childrens_Clothes,4.500000
Books_General_Interest,4.439421
Books_Imported,4.400000
Costruction_Tools_Tools,4.359223
...,...
Fashion_Male_Clothing,3.621212
Office_Furniture,3.484421
Home_Comfort_2,3.366667


# we will perform cohort analysis on customers binning them into groups and studying their behaviour across the months 

In [153]:

cust_ord_item_prod_sell['Sales']=cust_ord_item_prod_sell['order_item_id']*cust_ord_item_prod_sell['price']

In [154]:
a = cust_ord_item_prod_sell[['customer_unique_id','order_id','product_id','seller_id','Sales','order_purchase_timestamp']].\
sort_values(by='order_purchase_timestamp')

In [155]:
a['order_purchase_date']=pd.to_datetime(a['order_purchase_timestamp'].dt.date)

In [156]:
min_dates= a.groupby(by='customer_unique_id').agg(min_purchase_date=('order_purchase_date','min'))

In [157]:
df = pd.merge(left=a,right=min_dates,on='customer_unique_id',how='left')

In [158]:
df['order_period'] = df.order_purchase_date.apply(lambda x:x.strftime('%Y-%m'))

In [159]:
df['cohort_group'] = df.min_purchase_date.apply(lambda x:x.strftime('%Y-%m'))

In [160]:
df.head()

,customer_unique_id,order_id,product_id,seller_id,Sales,order_purchase_timestamp,order_purchase_date,min_purchase_date,order_period,cohort_group
0,b7d76e111c89f7ebf14761390f0f7d17,2e7a8482f6fb09756ca50c10d7bfc047,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,65.80,2016-09-04 21:15:00,2016-09-04,2016-09-04,2016-09,2016-09
1,b7d76e111c89f7ebf14761390f0f7d17,2e7a8482f6fb09756ca50c10d7bfc047,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,39.99,2016-09-04 21:15:00,2016-09-04,2016-09-04,2016-09,2016-09
2,4854e9b3feff728c13ee5fc7d1547e92,e5fa5a7210941f7d56d0208e4e071d35,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,59.50,2016-09-05 00:15:00,2016-09-05,2016-09-05,2016-09,2016-09
3,009b0127b727ab0ba422f6d9604487c7,809a282bbd5dbcabb6f2f724fca862ec,NaN,NaN,NaN,2016-09-13 15:24:00,2016-09-13,2016-09-13,2016-09,2016-09
4,830d5b7aaa3b6f1e9ad63703bec97d23,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,134.97,2016-09-15 12:16:00,2016-09-15,2016-09-15,2016-09,2016-09


In [161]:
cohorts = df.groupby(by=['cohort_group','order_period']).agg(Total_customers=('customer_unique_id','nunique'),Total_orders=('order_id','nunique'),\
Total_products=('product_id','count'),Total_sales=('Sales',np.sum))

In [162]:
cohorts

Total_customers  Total_orders  Total_products  \
cohort_group order_period                                                  
2016-09      2016-09                     4             4               6   
2016-10      2016-10                   321           324             363   
             2017-04                     1             1               1   
             2017-07                     1             1               1   
             2017-09                     1             1               1   
...                                    ...           ...             ...   
2018-08      2018-08                  6271          6316            7044   
             2018-09                     7             9               1   
             2018-10                     2             2               0   
2018-09      2018-09                     5             5               0   
2018-10      2018-10                     1             1               0   

                           Total_sales  
cohort_group order_period               
2016-09      2016-09            435.23  
2016-10      2016-10          56103.79  
             2017-04             99.99  
             2017-07            339.00  
             2017-09             49.00  
...                                ...  
2018-08      2018-08         927154.29  
             2018-09            145.00  
             2018-10              0.00  
2018-09      2018-09              0.00  
2018-10      2018-10              0.00  

[225 rows x 4 columns]

In [163]:
def cohort_period(df):
    df['cohort_period']=np.arange(len(df))+1
    return df
    

In [164]:
cohorts = cohorts.groupby(by='cohort_group').apply(cohort_period).droplevel(0)

In [165]:
cohorts.reset_index(inplace=True)

In [166]:
cohorts.set_index(['cohort_group','cohort_period'],inplace=True)

In [167]:
cohorts

order_period  Total_customers  Total_orders  \
cohort_group cohort_period                                               
2016-09      1                  2016-09                4             4   
2016-10      1                  2016-10              321           324   
             2                  2017-04                1             1   
             3                  2017-07                1             1   
             4                  2017-09                1             1   
...                                 ...              ...           ...   
2018-08      1                  2018-08             6271          6316   
             2                  2018-09                7             9   
             3                  2018-10                2             2   
2018-09      1                  2018-09                5             5   
2018-10      1                  2018-10                1             1   

                            Total_products  Total_sales  
cohort_group cohort_period                               
2016-09      1                           6       435.23  
2016-10      1                         363     56103.79  
             2                           1        99.99  
             3                           1       339.00  
             4                           1        49.00  
...                                    ...          ...  
2018-08      1                        7044    927154.29  
             2                           1       145.00  
             3                           0         0.00  
2018-09      1                           0         0.00  
2018-10      1                           0         0.00  

[225 rows x 5 columns]

# we will understand the customer retention over period of time for each cohorts

In [168]:
no_of_customers_of_group = cohorts.groupby(level=0)['Total_customers'].first()
no_of_customers_of_group.head(5)

cohort_group
2016-09       4
2016-10     321
2016-12       1
2017-01     764
2017-02    1752
Name: Total_customers, dtype: int64

In [169]:
cohorts['Total_customers'].unstack(level=0).divide(no_of_customers_of_group,axis=1).fillna(0)*100

cohort_group,2016-09,2016-10,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,...,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10
cohort_period,,,,,,,,,,,,,,,,,,,,,
1,100.0,100.000000,100.0,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0
2,0.0,0.311526,100.0,0.39267,0.228311,0.493171,0.595238,0.500556,0.477859,0.513611,...,0.341637,0.387537,0.459440,0.581135,0.528541,0.420875,0.724757,0.111625,0.0,0.0
3,0.0,0.311526,0.0,0.26178,0.285388,0.379363,0.212585,0.500556,0.350430,0.359527,...,0.384342,0.387537,0.315865,0.312919,0.271821,0.286195,0.032944,0.031893,0.0,0.0
4,0.0,0.311526,0.0,0.13089,0.114155,0.379363,0.170068,0.389321,0.414145,0.256805,...,0.284698,0.294528,0.287150,0.238415,0.211416,0.016835,0.000000,0.000000,0.0,0.0
5,0.0,0.311526,0.0,0.39267,0.399543,0.341426,0.340136,0.305895,0.254858,0.282486,...,0.284698,0.263525,0.129218,0.134108,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,0.0,0.311526,0.0,0.13089,0.114155,0.151745,0.255102,0.333704,0.382287,0.205444,...,0.156584,0.217021,0.129218,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,0.0,0.311526,0.0,0.52356,0.228311,0.151745,0.340136,0.417130,0.382287,0.308166,...,0.170819,0.201519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
8,0.0,0.623053,0.0,0.13089,0.171233,0.303490,0.297619,0.166852,0.223001,0.102722,...,0.227758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,0.0,0.623053,0.0,0.13089,0.171233,0.341426,0.297619,0.250278,0.127429,0.179764,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
